In [43]:

# using POMDPModels
# using POMDPSimulators
# using SARSOP
# using QMDP
# using BasicPOMCP
# using DMUStudent

using POMDPs
using QuickPOMDPs
using POMDPModelTools

using Random
rng = MersenneTwister(1234);

## 2D Discrete Representation Experiment

Frame the camera search problem in a 2D Discrete world to work out the kinks of a Julia POMDPs implementation.

In [2]:
# utility functions
function zoom_to_FOV(zoom)
    """This function is currently simple, but later may contain
    more complex calculations linking the zoom independent 
    (controlled) variable to other FOV paramters.
    """
    calc_focal_pt = zoom # just a 1-1 for now in this discrete world
    calc_FOV_width = 4 - zoom # an inverse relationship with hardcoded FOV max/min here
    return [calc_focal_pt, calc_FOV_width]
end

function clamp(n, smallest, largest)
    """Useful clamping function"""
    return max(smallest, min(n, largest))
end

clamp (generic function with 1 method)

In [3]:
# define a struct for the relevant problem state information
struct GW_state
    # camera state information
    cam_theta::Int # 1-36
    zoom::Int # 1-3
    focal_pt # 1-3
    FOV_width # 1-3
    
    # target state information
    target_theta # 1-36
    target_r # 1-3
    theta_dot # ....
    r_dot # 1-3
end

In [48]:
# 36*3*3*3*36*3*5*3
# if desicretize and make a list of 
# methods(observations)

# 1 method for generic function "observations":
[1] observations(s, a, sp) in Main at In[6]:4

In [49]:
struct InitDist end

function Base.rand(rng::AbstractRNG, d::InitDist)
    # generate a state
    i_cam_theta = rand(rng,(1, 36))
    i_zoom = rand(rng,(1, 3))
    i_focal_pt, i_FOV_width = zoom_to_FOV(i_zoom)
    
    # target state information
    i_target_theta = rand(rng,(1, 36))
    i_target_r = rand(rng,(1, 3))
    i_theta_dot = 0
    i_r_dot = 0
    
    init_state = GW_state(
        i_cam_theta, i_zoom, i_focal_pt, i_FOV_width,
        i_target_theta, i_target_r, i_theta_dot, i_r_dot,
    )
    return init_state 
end

In [64]:
function gen(s, a, rng)
    # generate a named tuple with :sp pointing to the new state
    
    # camera transitions
    #############################################################
    # need to be a function of zoom to account for speed decrease
    # at deep zoom. write up include pix / meter / second at zooms
    # aka the below says that the camera can pan faster when it
    # is zoomed out, which matches real physics
    sp_cam_theta = s.cam_theta
    
    if a == :left
        if s.zoom <= 2
            sp_cam_theta += 2
        elseif s.zoom > 2
            sp_cam_theta += 1
        end
    elseif a == :stay
        # do nothing
    elseif a == :right
        if s.zoom <= 2
            sp_cam_theta -= 2
        elseif s.zoom > 2
            sp_cam_theta -= 1
        end
    else
        # do nothing
    end

    # handle wrap around
    sp_cam_theta = sp_cam_theta % 36

    sp_zoom = s.zoom
    # update zoom
    if a == :zoom_in
        sp_zoom += 1
    elseif a == :stay_zoom
        # do nothing
    elseif a == :zoom_out
        sp_zoom -= 1
    end
    sp_zoom = clamp(sp_zoom, 1, 3)

    # update dependent state variables
    sp_focal_pt, sp_FOV_width = zoom_to_FOV(sp_zoom)
    
    # target transitions
    #############################################################   
    
    sp_target_theta = s.target_theta
    sp_target_r = s.target_r
    sp_theta_dot = s.theta_dot
    sp_r_dot = s.r_dot
    
    moving_target = false
    if moving_target
        # if want a moving target then add noise to
        # velocity to create more realistic motion
        
        # center at 0 mean
        sp_theta_dot += rand(rng) - 0.5 
        sp_r_dot += rand(rng) - 0.5 
        
        # clip to prevent crazy vels
        sp_theta_dot = clamp(sp_theta_dot, -1.9, 1.9) 
        sp_r_dot = clamp(sp_r_dot, -1, 1) 
        
    else
        # stationary target
        sp_theta_dot = 0
        sp_r_dot = 0
    end

    # update target state
    sp_target_theta += Int(sp_theta_dot)
    sp_target_theta = sp_target_theta % 36
    sp_target_r += Int(sp_r_dot)
    sp_target_r = clamp(sp_target_r, 1,3)

    new_state = GW_state(
        sp_cam_theta, sp_zoom, sp_focal_pt, sp_FOV_width,
        sp_target_theta, sp_target_r, sp_theta_dot, sp_r_dot,
    )
    return (sp=new_state,)
end

gen (generic function with 1 method)

In [6]:
# struct ObsDist
    
# end

# Base.rand()
# POMDPs.pdf()

function observations(s, a, sp)
    # should return a POMDPModelTools.BoolDistribution

    """Returns a boolean observation depending on observation
    model probabilities that are state dependent.
    """
    # if in FOV 
    # TODO: add handling wrap around to this check
    if abs(s.cam_theta - s.target_theta) <= ceil(s.FOV_width/2)
        # observation based on range
        if s.target_r == s.focal_pt 
            # in FOV and in focal range
            # 90% True Positive detection in this case
            return POMDPModelTools.BoolDistribution(0.9)
        else 
            # in FOV and not at correct focal range
            # 40% True Positive detection in this case
            return POMDPModelTools.BoolDistribution(0.4)
        end
    else
        # if not in FOV
        # 5% False Positive rate in this case
        return POMDPModelTools.BoolDistribution(0.05)
    end
    # return MvNormal(mean, cov) # Distributions.jl
end

observations (generic function with 1 method)

In [7]:
function actions()
    # should return a vector of actions available at the state
    A = [:left, :stay, :right, :zoom_in, :stay_zoom, :zoom_out, :track]
    return A
end

actions (generic function with 1 method)

In [67]:
function reward(s, a, sp, anything)
    """Return a positive reward if the action is to "Track"
    and a target is in the FOV.
    Track = begin a track since a target is confidently detected.
    Return a negative reward if the action is to "Track" but
    the target is not in the FOV.
    """

    if a == :track
        # if in FOV 
        # TODO: add handling wrap around to this check
        if abs(s.cam_theta - s.target_theta) <= ceil(s.FOV_width/2)
            return 10.0 # and terminal state
        else
            return -1.0
        end
    else
        return 0.0
    end
end

reward (generic function with 2 methods)

In [69]:
# define the QuickPOMDP
m = QuickPOMDP(
#     states=GW_state,
    statetype=GW_state,
    gen=gen,
    # initialstate=initialstate,
    initialstate_distribution=InitDist(),
    observation=observations,
#     obstype=Array / Tuple{Bool, Float}, # for multi parameter observation return
    actions=actions,
    reward=reward,
)


┌ Warning: Unable to infer observation type for a QuickPOMDP; using Any. This may have significant performance consequences. Use the obstype keyword argument to specify a concrete observation type.
└ @ QuickPOMDPs /home/mike/.julia/packages/QuickPOMDPs/lfE03/src/quick.jl:168


QuickPOMDP{UUID("99bd886c-cc61-44f6-89a2-d932d6111326"),GW_state,Symbol,Any,NamedTuple{(:isterminal, :actionindex, :statetype, :reward, :gen, :actions, :discount, :initialstate, :initialstate_distribution, :observation),Tuple{Bool,Dict{Symbol,Int64},DataType,typeof(reward),typeof(gen),typeof(actions),Float64,QuickPOMDPs.var"#17#21"{Dict{Symbol,Any}},InitDist,typeof(observations)}}}((isterminal = false, actionindex = Dict(:left => 1,:right => 3,:stay => 2,:zoom_out => 6,:stay_zoom => 5,:track => 7,:zoom_in => 4), statetype = GW_state, reward = reward, gen = gen, actions = actions, discount = 1.0, initialstate = QuickPOMDPs.var"#17#21"{Dict{Symbol,Any}}(Dict{Symbol,Any}(:isterminal => false,:actionindex => Dict(:left => 1,:right => 3,:stay => 2,:zoom_out => 6,:stay_zoom => 5,:track => 7,:zoom_in => 4),:statetype => GW_state,:reward => reward,:gen => gen,:actions => actions,:discount => 1.0,:initialstate => QuickPOMDPs.var"#17#21"{Dict{Symbol,Any}}(#= circular reference @-2 =#),:initialst

In [16]:
using POMDPModels
using QMDP
solver = QMDPSolver()

QMDPSolver{DiscreteValueIteration.ValueIterationSolver}(DiscreteValueIteration.ValueIterationSolver(100, 0.001, false, true, Float64[]))

In [ ]:
qmdp_policy = solve(solver, m)

In [29]:
using POMDPPolicies # For creating a random policy

In [85]:
# Create a policy that chooses actions at random
rand_policy = RandomPolicy(m);

In [86]:
simple_policy = FunctionPolicy(
    function(o) # how do you give the function policy the PO state too? 
#         print(o)
#         print(s)
        if o == true
            return :track
        elseif rand() < 0.8
            return :left
        else
            return :zoom_out
        end
    end
)

FunctionPolicy{var"#11#12"}(var"#11#12"())

In [91]:
# Create and run the rollout simulator
using POMDPSimulators
rollout_sim = RolloutSimulator(max_steps=200);

In [93]:
# fib_reward = simulate(rollout_sim, m, fib_policy);
rand_reward = simulate(rollout_sim, m, rand_policy);
simp_reward = simulate(rollout_sim, m, simple_policy);

In [94]:

# @show fib_reward;
@show rand_reward;
@show simp_reward;

rand_reward = 41.0
simp_reward = 641.0


In [ ]:
# POWCPOW could work with this setup
# solvers like QMDP and FIB require tabularized everything possible.. 
# POMCP

# per

# 

# TODO
- define terminal state
- run MC sims on random and simple to show working
- figure out how to get the observable part of the state into the simple heuristic function?
- try out various solvers and try to find one that will work on this definition. Eg. POMCPOW. POMCP. maybe QMDP.
- I think I'll need to add a belief updater to those ^^ algs to be able to use them

# Questions
- How to manually update the belief? 
- How is a julia pomdps policy parameterized/stored? 

# Shelf
- Adding extra parameters in the observation. In real life, could have that be a heuristic chunk of the search problem. Damn. Would've been so cool to really build this system out for real... the D4D system.
